### 1. ¿Qué es Statsmodels?
`statsmodels` es una librería de Python específicamente diseñada para la **econometría y la estadística descriptiva**. A diferencia de *scikit-learn* (que se enfoca en el aprendizaje automático y la predicción), *statsmodels* se enfoca en la **inferencia**: entender la relación entre las variables y la validez estadística del modelo.

### 2. ¿Por qué se importa como `sm`?
Es simplemente una convención (un "alias"). Casi todos los científicos de datos usan `sm` para escribir menos código. El módulo `api` contiene las herramientas principales, como la Regresión Lineal (OLS) y la Regresión Logística (Logit).

### 3. ¿Qué lo hace especial? (El "Summary")
La mayor ventaja de usar `statsmodels` es que, tras entrenar un modelo, puedes ejecutar el comando `.summary()`. Este comando te devuelve una tabla detallada que incluye:
*   **P-values (P>|z|):** Para saber si una variable es realmente significativa o es puro azar.
*   **Coeficientes:** El impacto exacto de cada variable.
*   **Intervalos de confianza:** El rango en el que se encuentra el valor real del parámetro.
*   **Métricas de calidad:** Como el **AIC**, BIC y R-cuadrado.

### 4. Diferencia clave: El Intercepto (La Constante)
Esta es la diferencia técnica más importante que debes recordar:
*   **Scikit-learn:** Añade automáticamente una constante (el punto donde la línea corta el eje Y) al modelo.
*   **Statsmodels:** **No la añade por defecto.** Tienes que usar la función `sm.add_constant(X)` manualmente. Si no lo haces, estarás obligando a tu modelo a pasar por el punto (0,0), lo cual suele ser un error estadístico.

### 5. Ejemplo de uso básico
Aquí ves cómo se estructura típicamente un análisis con esta librería:

```python
import statsmodels.api as sm

# 1. Preparar las variables (X e y)
X = df[['edad', 'ingresos']]
y = df['compro_producto']

# 2. IMPORTANTE: Añadir la constante (intercepto)
X = sm.add_constant(X)

# 3. Crear y entrenar el modelo (Logit para regresión logística)
modelo = sm.Logit(y, X).fit()

# 4. Ver la "magia" estadística
print(modelo.summary())
```

### Resumen: Statsmodels vs Scikit-learn
| Característica | Scikit-learn (`sklearn`) | Statsmodels (`sm`) |
| :--- | :--- | :--- |
| **Objetivo** | Predicción y Machine Learning. | Inferencia y Estadística. |
| **Facilidad** | Muy fácil para flujos de producción. | Un poco más manual (R-style). |
| **Información** | Solo te da la predicción. | Te da p-values, errores estándar, etc. |
| **Intercepto** | Automático. | Manual (`add_constant`). |

**¿Cuándo usar `sm`?** Úsalo cuando necesites **explicar** por qué sucede algo (ej. "¿Cómo influye la educación en el salario?"), mientras que usa `sklearn` cuando solo te importe la **precisión** del resultado final (ej. "¿Cuánto ganará esta persona mañana?").

---

Para variar el enfoque, en lugar de una clasificación (Sí/No), utilizaremos una **Regresión Lineal Múltiple (OLS - Ordinary Least Squares)**.

Este ejemplo es ideal para entender cómo diferentes factores influyen en una variable continua, como el **precio de una vivienda**.

### Escenario: Estimación del precio de alquiler
Imagina que trabajas para una inmobiliaria y quieres saber si el **tamaño del apartamento**, la **edad del edificio** y la **distancia al centro** realmente afectan el precio del alquiler.

```python
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. Crear datos sintéticos
np.random.seed(42)
n = 100

# Variables independientes (X)
metros_cuadrados = np.random.normal(80, 20, n)  # Promedio 80m2
edad_edificio = np.random.randint(1, 50, n)     # De 1 a 50 años
distancia_centro = np.random.uniform(0.5, 10, n) # De 0.5 a 10 km

# Variable dependiente (y) - El precio tiene una lógica:
# + precio por metro, - precio por año de antigüedad, - precio por distancia
precio_alquiler = (20 * metros_cuadrados) - (5 * edad_edificio) - (50 * distancia_centro) + np.random.normal(0, 100, n)

df = pd.DataFrame({
    'Metros': metros_cuadrados,
    'Edad': edad_edificio,
    'Distancia': distancia_centro,
    'Precio': precio_alquiler
})

# 2. Definir variables
X = df[['Metros', 'Edad', 'Distancia']]
y = df['Precio']

# 3. ¡PASO CRUCIAL EN STATSMODELS!: Añadir la constante (intercepto)
# Sin esto, el modelo asume que si las X son 0, el precio es 0.
X = sm.add_constant(X)

# 4. Ajustar el modelo OLS (Ordinary Least Squares)
modelo_lineal = sm.OLS(y, X).fit()

# 5. Mostrar el resumen estadístico
print(modelo_lineal.summary())
```

### ¿Qué información nueva obtenemos aquí? (Diferente a la logística)

Al ver el `summary()`, debes fijarte en tres puntos que `statsmodels` calcula magistralmente:

1.  **R-squared (R-cuadrado):** Nos dice qué porcentaje de la variación del precio explica nuestro modelo. Si es 0.85, significa que el tamaño, la edad y la distancia explican el 85% del precio del alquiler.
2.  **Coef (Coeficientes):** 
    *   Si el coeficiente de **Metros** es `19.5`, significa que por cada metro cuadrado extra, el alquiler sube en promedio \$19.5.
    *   Si el coeficiente de **Distancia** es `-48.2`, significa que por cada kilómetro que te alejas del centro, el precio baja \$48.2.
3.  **Prob (F-statistic):** Es el p-valor global del modelo. Si es menor a 0.05, significa que el modelo es estadísticamente significativo (no es fruto de la casualidad).
4.  **Durbin-Watson:** Una métrica que aparece en el resumen y sirve para detectar si hay correlación entre los errores (autocorrelación), algo vital en análisis de datos económicos.

### ¿Cuándo usar este ejemplo en lugar del anterior?
*   Usa el de **Regresión Logística** (el anterior) cuando quieras predecir **categorías** (Falla/No falla, Compra/No compra).
*   Usa este de **Regresión Lineal (OLS)** cuando quieras predecir un **valor numérico exacto** (Precio, Temperatura, Ventas, Peso).